In [1]:
## Linkdin Content Writer

In [2]:
# Import the libraries

from crewai import Agent, Task, Crew
from crewai.tools import tool
from crewai import LLM
from docx import Document
import os
import requests

In [3]:
# Get the keys
pse_key = os.getenv('API_KEY')
model_api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
# Lets define the tools that we need

# Step1: Search for web for topics.
# Step2: Craete a file and save it in the local system

In [5]:
@tool("web_search")
def web_search(query: str) -> str:
    
    """Perform Google Custom Search and return top result snippet."""

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": pse_key,
        "cx": '84da54018bf7d48ed',
        "q": query,
        "num": 1
    }
    response = requests.get(url, params=params).json()
    items = response.get("items", [])
    if items:
        return items[0].get("snippet", "No snippet available.")
    return "No results found."

In [6]:
@tool('file_saver')
def create_file(filename: str, content: str) -> str:
    """
    Create and save a .docx file locally with the given filename and content.
    Save only the final output from editor to a .docx file.
    """
    try:
        # Ensure directory exists
        os.makedirs("generated_files", exist_ok=True)
        
        # Ensure the filename has .docx extension
        if not filename.endswith(".docx"):
            filename += ".docx"
        
        file_path = os.path.join("generated_files", filename)
        
        # Create a Word document and add content
        doc = Document()
        doc.add_paragraph(content)
        doc.save(file_path)
        
        return f"File '{filename}' created successfully at: {os.path.abspath(file_path)}"
    
    except Exception as e:
        return f"Failed to create file '{filename}': {str(e)}"


In [7]:
# Configure LLM Model

llm_model = LLM(model='gemini/gemini-2.0-flash',
    api_key=model_api_key,
    temperature=0.7)

In [8]:
# Create Agents
researcher = Agent(
    role="Researcher",
    goal="Find relevant facts and stats about the topic. Use web search tool to find latest information",
    backstory="Expert in online research and data gathering.",
    tools=[web_search],
    verbose=True,
    llm=llm_model)

writer = Agent(
    role="Writer",
    goal="Write a LinkedIn-style post from research summary",
    backstory="Professional content writer skilled in engaging tone.",
    verbose=True,
    llm=llm_model)


editor = Agent(
    role="Editor",
    goal="Polish and optimize post for LinkedIn tone and engagement",
    backstory="Experienced social media editor.",
    verbose=True,
    llm=llm_model)


saver = Agent(
    role="File Saver",
    goal="Save the final edited post to a local .docx file.",
    backstory="Handles file creation and organization.",
    tools=[create_file],
    verbose=True,
    llm=llm_model)

In [ ]:
def AgenticAI_Output(topic: str):
    
    research_task = Task(
    name="Research Task",
    description=f'''For the given topic: {topic} use the web search tool to find the latest information.
    gather as much information as possible. Include the facts available over the web on the given topic.''',
    expected_output="A 2 to 3 page full detailed research report.",
    agent=researcher)

    write_task = Task(name='Write Task',
    description=f'''Draft a LinkedIn post based on the research summary. Include a strong hook and call-to-action.
    Here is the web reseaqrch report : {research_task.output}''',
    expected_output="A well-structured LinkedIn post draft in engaging and professional tone.",
    agent=writer,
    depends_on=[research_task])

    edit_task = Task(name='Edit Task',
    description=f'''Refine and optimize the LinkedIn post for clarity, engagement, and LinkedIn formatting.
    here is the first draft of the post: {write_task.output}''',
    expected_output="Final polished LinkedIn post ready for publishing with proper hashtags and emojis.",
    agent=editor,
    depends_on=[write_task])


    save_task = Task(
    name="Save Task",
    description=f'''use {topic}_post.docx as filename and save the end and polished result given by
    {edit_task.output} as content for docx word file. Do not save outputs of all the agents, only save the output of edit_task agent''',
    expected_output=".docx file conatining the output of edit_task",
    agent=saver,
    depends_on=[edit_task])

    crew = Crew(
    agents=[researcher,writer, editor ,saver],
    tasks=[research_task,write_task,edit_task,save_task], 
    verbose=True)

    result = crew.kickoff(inputs={"topic": topic})
    return result


    



In [10]:
AgenticAI_Output('Future of E20 fuel in indian market')

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ac76725a-b825-4761-bfb2-1ba6232e133e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: For the given topic: Future of E20 fuel in indian market use the web search tool to find the latest      │
│  information.                                                                                                   │
│      gather as much information as possible. Include the facts available over the web on the given topic.       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Thought: Okay, I understand. I need to create a detailed research report (2-3 pages) on the future of E20      │
│  fuel in the Indian market using the web search tool to gather as much information as possible, including       │
│  available facts and statistics.                                                                                │
│                                                                                                                 │
│  Using Tool: web_search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"E20 fuel India future\"}"                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: File Saver                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ⛽️ India's E20 Fuel Revolution: A Step Towards a Sustainable Future? 🇮🇳                                        │
│                                                                                                                 │
│  The Indian government is accelerating its push for 20% ethanol-blended gasoline (E20) by 2025-26! 🗓️ But what   │
│  does this mean for our environment, economy, and even our vehicles? 🚗💨                                       │
│                                                                                                                 │
│  Here's a quick breakdown from a detailed report:                                                               │
│                                                                                                                 │
│  ✅ **The Good:**                                                                                               │
│  *   Reduced import dependence ⬇️ (saving precious foreign exchange!)                                            │
│  *   Lower emissions (CO, HC, and SO2) 🌿                                                                       │
│  *   Boost to the agricultural sector 🌾                                                                        │
│  *   Improved air quality 💨                                                                                    │
│                                                                                                                 │
│  ⚠️ **The Challenges:**                                                                                          │
│  *   Scaling up ethanol production 🏭                                                                           │
│  *   Managing water resources 💧                                                                                │
│  *   Ensuring food security 🍽️                                                                                   │
│  *   Addressing vehicle compatibility issues (especially for older models) 🛠️                                    │
│                                                                                                                 │
│  🚗 **Impact on Your Ride:** Newer vehicles are generally E20-ready, but older ones might face corrosion and    │
│  reduced fuel efficiency. Fuel additives and part replacements could be the answer. 🤔                          │
│                                                                                                                 │
│  🌱 **Environmental Impact:** While E20 promises reduced emissions, sustainable ethanol production is crucial   │
│  to avoid negative impacts like deforestation and water pollution. 🌳➡️ 🏞️                                        │
│                                                                                                                 │
│  🤔 **The Big Question:** Can India successfully navigate these challenges to unlock the full potential of E20  │
│  fuel?                                                                                                          │
│                                                                                                                 │
│  ➡️ **What are your thoughts on the E20 initiative? Share your insights and concerns in the comments below!**    │
│  Let's discuss the future of fuel in India! 💬👇              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: Save Task                                                                                                │
│  Agent: File Saver                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ac76725a-b825-4761-bfb2-1ba6232e133e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ⛽️ India's E20 Fuel Revolution: A Step Towards a Sustainable Future? 🇮🇳                          │
│                                                                                                                 │
│  The Indian government is accelerating its push for 20% ethanol-blended gasoline (E20) by 2025-26! 🗓️ But what   │
│  does this mean for our environment, economy, and even our vehicles? 🚗💨                                       │
│                                                                                                                 │
│  Here's a quick breakdown from a detailed report:                                                               │
│                                                                                                                 │
│  ✅ **The Good:**                                                                                               │
│  *   Reduced import dependence ⬇️ (saving precious foreign exchange!)                                            │
│  *   Lower emissions (CO, HC, and SO2) 🌿                                                                       │
│  *   Boost to the agricultural sector 🌾                                                                        │
│  *   Improved air quality 💨                                                                                    │
│                                                                                                                 │
│  ⚠️ **The Challenges:**                                                                                          │
│  *   Scaling up ethanol production 🏭                                                                           │
│  *   Managing water resources 💧                                                                                │
│  *   Ensuring food security 🍽️                                                                                   │
│  *   Addressing vehicle compatibility issues (especially for older models) 🛠️                                    │
│                                                                                                                 │
│  🚗 **Impact on Your Ride:** Newer vehicles are generally E20-ready, but older ones might face corrosion and    │
│  reduced fuel efficiency. Fuel additives and part replacements could be the answer. 🤔                          │
│                                                                                                                 │
│  🌱 **Environmental Impact:** While E20 promises reduced emissions, sustainable ethanol production is crucial   │
│  to avoid negative impacts like deforestation and water pollution. 🌳➡️ 🏞️                                        │
│                                                                                                                 │
│  🤔 **The Big Question:** Can India successfully navigate these challenges to unlock the full potential of E20  │
│  fuel?                                                                                                          │
│                                                                                                                 │
│  ➡️ **What are your thoughts on the E20 initiative? Share yo

CrewOutput(raw="⛽️ India's E20 Fuel Revolution: A Step Towards a Sustainable Future? 🇮🇳\n\nThe Indian government is accelerating its push for 20% ethanol-blended gasoline (E20) by 2025-26! 🗓️ But what does this mean for our environment, economy, and even our vehicles? 🚗💨\n\nHere's a quick breakdown from a detailed report:\n\n✅ **The Good:**\n*   Reduced import dependence ⬇️ (saving precious foreign exchange!)\n*   Lower emissions (CO, HC, and SO2) 🌿\n*   Boost to the agricultural sector 🌾\n*   Improved air quality 💨\n\n⚠️ **The Challenges:**\n*   Scaling up ethanol production 🏭\n*   Managing water resources 💧\n*   Ensuring food security 🍽️\n*   Addressing vehicle compatibility issues (especially for older models) 🛠️\n\n🚗 **Impact on Your Ride:** Newer vehicles are generally E20-ready, but older ones might face corrosion and reduced fuel efficiency. Fuel additives and part replacements could be the answer. 🤔\n\n🌱 **Environmental Impact:** While E20 promises reduced emissions, sustainabl